# 1. Business Task
Cyclistic is a company looking for how to keep growing in the future. Their business is based on different bike rental plans:

-Single-ride pass.

-Full-day pass.

-Annual membership.

We are going to analyse how annual members (Annual membership) differ from casual riders (Single pass and Full-day pass) using Cyclistic, in order to help the organization design marketing strategies aimed at converting casual riders into annual members. 

Annual members are more profitable than casual riders, so stakeholders believe that increasing the amount of annual riders is key for Cyclistic's growth in the future.

# 2. Data Preparation

The data used is internally collected monthly data from 2013 to 2022 by Cyclistic from their users. The data is stored by the company on AWS' S3 cloud object storaged solution in dot zip files.

https://divvy-tripdata.s3.amazonaws.com/index.html

In general the data contains anonymous user information regarding:

-ride ID

-starting station

-ending station

-user type (casual rider or annual member)

It's stored in the structured data format. However, data was not saved the same throught the years.










### Cleaning the data
There's some cleaning and formatting that needs to be done to the data because of the following issues:

-There's columns missing from one time period to the other, such as: 

user gender, birthyear, start_lat, start_lng, end_lat, end_lng, rideable_type, bikeid, tripduration.

-The following columns are the same although they have different names:

ride_id == trip_id

starttime == started_at

stoptime == ended_at

from_station_name == start_station_name

to_station_name == end_station_name

from_station_id == start_station_id

to_station_id == end_station_id


#### Data cleaning and formatting with Python and Pandas
The following changes were performed to the data